# Segmenting and Clustering Neighborhoods in Toronto

In this exercise we use a web scraping technique to create a dataframe of the neighborhoods in Toronto grouped by postal code

## Import required libraries
For this exrecise we'll use 'BeautifulSoup' for web scraping and 'requests' to get to data from the Wikepdia webpage located at <https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M>

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import csv
import folium
import matplotlib
from pandas.io.json import json_normalize
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

## Get data from website

First we'll get the data from Wikipedia

In [2]:
#Get Data from website
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
source[0:3000]

'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>List of postal codes of Canada: M - Wikipedia</title>\n<script>document.documentElement.className=document.documentElement.className.replace(/(^|\\s)client-nojs(\\s|$)/,"$1client-js$2");RLCONF={"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":890001695,"wgRevisionId":890001695,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgBreakFrames":!1,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","Au

## Parse website data
Then we'll parse the data retrived from the Wikipedia

In [3]:
#Parse data
soup = BeautifulSoup(source,'lxml')
table = soup.find('table').text
table_split = table.split('\n\n')
table_split[0:10]

['',
 'Postcode\nBorough\nNeighbourhood',
 '\nM1A\nNot assigned\nNot assigned',
 '\nM2A\nNot assigned\nNot assigned',
 '\nM3A\nNorth York\nParkwoods',
 '\nM4A\nNorth York\nVictoria Village',
 '\nM5A\nDowntown Toronto\nHarbourfront',
 '\nM5A\nDowntown Toronto\nRegent Park',
 '\nM6A\nNorth York\nLawrence Heights',
 '\nM6A\nNorth York\nLawrence Manor']

## Create DataFrame and Clean Data
Once the data is parsed we'll put it into a Pandas dataframe and clean the data

In [4]:
#create new df 
df = pd.DataFrame({'col':table_split})

#split column of data into multiple columns
df = df['col'].apply(lambda x : pd.Series(x.split('\n')))

#Get column names
col_names = df.iloc[1]

#Drop rows with empty values
df.dropna(inplace=True)

#Drop empty columns
df.drop([0], axis=1, inplace=True)

#Reshape columns
df.columns = range(df.shape[1])

#Drop 'Not assigned' Burough rows
df = df[df[1].str.contains('Not assigned') == False]

#Rename 'Not assigned' Neighborhoods
na_location = df[df[2].str.contains('Not assigned') == True].index[0]
df[2].loc[na_location] = df[1].loc[na_location]

#Add column names
df.rename(columns=col_names, inplace=True)

#Reset row index
df.reset_index(drop=True, inplace=True)

df.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


## Group data
Now let's group the data my postcode

In [5]:
#Group by postcode

df_grp = pd.DataFrame(df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda tags: ', '.join(tags)))
df_grp.reset_index(inplace=True)
df_grp.head(10)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


## DataFrame size

Now that the dataframe has been cleaned and grouped by postcode, let's see the datafram size

In [6]:
df_grp.shape

(103, 3)

As shown above, the dataframe has 3 columns and 103 rows.

In [7]:
#Create DataFrame from csv file

csv_file = open('./Geospatial_Coordinates.csv', 'r')

lat_long = csv.reader(csv_file, delimiter=',')
#df_coord = pd.DataFrame({'PostalCode', 'Lat', 'Long'})
df_coord = pd.DataFrame()
i=0
for row in lat_long:
    df_coord[i] = row
    i = i+1

csv_file.close()

df_coord = df_coord.transpose()

new_header = df_coord.iloc[0] #grab the first row for the header
df_coord = df_coord[1:] #take the data less the header row
df_coord.columns = new_header #set the header row as the df header

df_coord = df_coord.set_index('Postal Code')
df_coord.head()

,Latitude,Longitude
Postal Code,,
M1B,43.8066863,-79.1943534
M1C,43.7845351,-79.1604971
M1E,43.7635726,-79.1887115
M1G,43.7709921,-79.2169174
M1H,43.773136,-79.2394761


In [8]:
#Join df_grp and df_coord to form df_latlong

df_grp = df_grp.set_index('Postcode')

df_grp.join(df_coord)

df_latlong = df_grp.join(df_coord)

#Reset index
df_latlong.reset_index(drop=False, inplace=True)

df_latlong.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.8066863,-79.1943534
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.7845351,-79.1604971
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7635726,-79.1887115
3,M1G,Scarborough,Woburn,43.7709921,-79.2169174
4,M1H,Scarborough,Cedarbrae,43.773136,-79.2394761
5,M1J,Scarborough,Scarborough Village,43.7447342,-79.2394761
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.7279292,-79.2620294
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.7111117,-79.2845772
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.2394761
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.2648481


In [9]:
df_latlong.shape

(103, 5)

## Create a map of Toronto using Lat Long Values

In [10]:
latitude = 43.723963
longitude = -79.387207

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_latlong['Latitude'].astype(float), df_latlong['Longitude'].astype(float), df_latlong['Neighbourhood']):
    
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)
    
map_toronto

## Define Foursquare Credentials and Version

In [41]:
CLIENT_ID = 'X' # your Foursquare ID
CLIENT_SECRET = 'X' # your Foursquare Secret
VERSION = '20190530' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: X
CLIENT_SECRET:X


## Explore a neighborhood in the DataFrame

In [12]:
df_latlong.loc[5, 'Neighbourhood']

'Scarborough Village'

In [13]:
neighborhood_latitude = df_latlong.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_latlong.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_latlong.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rouge, Malvern are 43.8066863, -79.1943534.


## Top 100 venues in this neighborhood within 500 meters

In [14]:
LIMIT = 100
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [15]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5cf41c53f594df57eebac2f2'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 43.811186304500005,
    'lng': -79.1881295807304},
   'sw': {'lat': 43.8021862955, 'lng': -79.20057721926959}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb6b9446edc76b0d771311c',
       'name': "Wendy's",
       'location': {'crossStreet': 'Morningside & Sheppard',
        'lat': 43.80744841934756,
        'lng': -79.19905558052072,
        'labeledLatLngs': [{'label': 'dis

## Extract categories of the venues

In [16]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [17]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Wendy's,Fast Food Restaurant,43.807448,-79.199056
1,Interprovincial Group,Print Shop,43.805630,-79.200378


In [18]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

2 venues were returned by Foursquare.


## Explore neighborhoods in Toronto

In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [20]:
toronto_venues = getNearbyVenues(names=df_latlong['Neighbourhood'],
                                   latitudes=df_latlong['Latitude'],
                                   longitudes=df_latlong['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

In [21]:
print(toronto_venues.shape)
toronto_venues.head()

(2256, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.8066863,-79.1943534,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge, Malvern",43.8066863,-79.1943534,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Highland Creek, Rouge Hill, Port Union",43.7845351,-79.1604971,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Highland Creek, Rouge Hill, Port Union",43.7845351,-79.1604971,Affordable Toronto Movers,43.787919,-79.162977,Moving Target
4,"Guildwood, Morningside, West Hill",43.7635726,-79.1887115,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place


In [22]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",3,3,3,3,3,3
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",10,10,10,10,10,10
"Alderwood, Long Branch",8,8,8,8,8,8
"Bathurst Manor, Downsview North, Wilson Heights",18,18,18,18,18,18
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",26,26,26,26,26,26
Berczy Park,56,56,56,56,56,56


In [23]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 282 uniques categories.


## Analyze Each Neighborhood

In [24]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
toronto_onehot.shape

(2256, 282)

In [26]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.010000,0.000000,0.000000,0.000000,0.0000,0.010000,0.0,0.000000,0.000000
1,Agincourt,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.000000
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.000000
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.000000
4,"Alderwood, Long Branch",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.000000
5,"Bathurst Manor, Downsview North, Wilson Heights",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.000000,0.055556,0.000000,0.0000,0.000000,0.0,0.000000,0.000000
6,Bayview Village,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.000000
7,"Bedford Park, Lawrence Manor East",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.038462
8,Berczy Park,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.017857,0.000000,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.000000
9,"Birch Cliff, Cliffside West",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.000000


In [27]:
toronto_grouped.shape

(100, 282)

## Print each neighborhood with top 5 most common venues

In [28]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2      Thai Restaurant  0.04
3                  Bar  0.04
4  American Restaurant  0.04


----Agincourt----
            venue  freq
0          Lounge  0.25
1    Skating Rink  0.25
2  Breakfast Spot  0.25
3  Clothing Store  0.25
4     Music Store  0.00


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                venue  freq
0                Park  0.67
1          Playground  0.33
2         Men's Store  0.00
3       Metro Station  0.00
4  Mexican Restaurant  0.00


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                 venue  freq
0        Grocery Store   0.2
1          Coffee Shop   0.1
2  Fried Chicken Joint   0.1
3          Pizza Place   0.1
4             Pharmacy   0.1


----Alderwood, Long Branch----
          venue  freq
0   Pizza Place  0.25
1           Gym  0.12
2

                venue  freq
0   Convenience Store   0.2
1    Department Store   0.2
2      Discount Store   0.2
3  Chinese Restaurant   0.2
4         Coffee Shop   0.2


----East Toronto----
                        venue  freq
0                        Park  0.50
1           Convenience Store  0.25
2               Metro Station  0.25
3  Modern European Restaurant  0.00
4                      Museum  0.00


----Emery, Humberlea----
            venue  freq
0  Baseball Field   1.0
1     Yoga Studio   0.0
2  Medical Center   0.0
3     Music Store   0.0
4          Museum   0.0


----Fairview, Henry Farm, Oriole----
                  venue  freq
0        Clothing Store  0.15
1           Coffee Shop  0.07
2  Fast Food Restaurant  0.07
3                Bakery  0.03
4   Japanese Restaurant  0.03


----First Canadian Place, Underground city----
          venue  freq
0          Café  0.09
1   Coffee Shop  0.09
2         Hotel  0.04
3  Burger Joint  0.03
4    Restaurant  0.03


----Flemingdon Park,

                             venue  freq
0             Fast Food Restaurant   0.5
1                       Print Shop   0.5
2                      Yoga Studio   0.0
3  Molecular Gastronomy Restaurant   0.0
4                      Music Store   0.0


----Runnymede, Swansea----
                venue  freq
0         Pizza Place  0.08
1         Coffee Shop  0.08
2                Café  0.08
3    Sushi Restaurant  0.06
4  Italian Restaurant  0.06


----Ryerson, Garden District----
                       venue  freq
0                Coffee Shop  0.10
1             Clothing Store  0.06
2                       Café  0.04
3             Cosmetics Shop  0.04
4  Middle Eastern Restaurant  0.03


----Scarborough Village----
                      venue  freq
0                Playground   0.5
1             Grocery Store   0.5
2  Mediterranean Restaurant   0.0
3               Men's Store   0.0
4             Metro Station   0.0


----St. James Town----
            venue  freq
0     Coffee Shop  0.07
1    

Put it in Pandas frame

In [29]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [30]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Thai Restaurant,Bar,Steakhouse,American Restaurant,Hotel,Cosmetics Shop,Bakery,Burger Joint
1,Agincourt,Clothing Store,Breakfast Spot,Lounge,Skating Rink,Electronics Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Playground,Eastern European Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Pizza Place,Coffee Shop,Beer Store,Liquor Store,Sandwich Place,Fried Chicken Joint,Fast Food Restaurant,Pharmacy,Cosmetics Shop
4,"Alderwood, Long Branch",Pizza Place,Pharmacy,Coffee Shop,Sandwich Place,Pub,Skating Rink,Gym,Empanada Restaurant,Electronics Store,Ethiopian Restaurant


## k-means clustering into 7 clusters

In [31]:
# set number of clusters
kclusters = 7

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=7).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 3, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [32]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
#print(neighborhoods_venues_sorted.shape)

toronto_merged = df_latlong

#merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood', how='inner')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.8066863,-79.1943534,1,Fast Food Restaurant,Print Shop,Women's Store,Eastern European Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.7845351,-79.1604971,1,Moving Target,Bar,Women's Store,Fish & Chips Shop,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7635726,-79.1887115,1,Spa,Intersection,Rental Car Location,Electronics Store,Mexican Restaurant,Breakfast Spot,Medical Center,Pizza Place,Gluten-free Restaurant,Event Space
3,M1G,Scarborough,Woburn,43.7709921,-79.2169174,1,Coffee Shop,Soccer Field,Korean Restaurant,Women's Store,Electronics Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.2394761,1,Athletics & Sports,Fried Chicken Joint,Thai Restaurant,Bakery,Bank,Caribbean Restaurant,Hakka Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant


In [33]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'].astype(float), toronto_merged['Longitude'].astype(float), toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster+1), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

Cluster 1

In [34]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Scarborough,0,Bakery,Bus Line,Fast Food Restaurant,Soccer Field,Intersection,Park,Bus Station,Dumpling Restaurant,Donut Shop,Drugstore
25,North York,0,Fast Food Restaurant,Food & Drink Shop,Park,Women's Store,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
30,North York,0,Electronics Store,Airport,Park,Women's Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
64,Central Toronto,0,Jewelry Store,Sushi Restaurant,Park,Trail,Women's Store,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore
72,North York,0,Pub,Asian Restaurant,Park,Japanese Restaurant,Eastern European Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
74,York,0,Park,Women's Store,Fast Food Restaurant,Pharmacy,Market,Gym,Dim Sum Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant
79,North York,0,Basketball Court,Construction & Landscaping,Park,Bakery,Empanada Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
90,Etobicoke,0,Smoke Shop,River,Pool,Park,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
91,Etobicoke,0,Baseball Field,Locksmith,Park,Women's Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
100,Etobicoke,0,Pizza Place,Bus Line,Park,Mobile Phone Shop,Eastern European Restaurant,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store


Cluster 2

In [35]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,1,Fast Food Restaurant,Print Shop,Women's Store,Eastern European Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
1,Scarborough,1,Moving Target,Bar,Women's Store,Fish & Chips Shop,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
2,Scarborough,1,Spa,Intersection,Rental Car Location,Electronics Store,Mexican Restaurant,Breakfast Spot,Medical Center,Pizza Place,Gluten-free Restaurant,Event Space
3,Scarborough,1,Coffee Shop,Soccer Field,Korean Restaurant,Women's Store,Electronics Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
4,Scarborough,1,Athletics & Sports,Fried Chicken Joint,Thai Restaurant,Bakery,Bank,Caribbean Restaurant,Hakka Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant
5,Scarborough,1,Playground,Grocery Store,Eastern European Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
6,Scarborough,1,Discount Store,Convenience Store,Department Store,Coffee Shop,Chinese Restaurant,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore
8,Scarborough,1,Motel,American Restaurant,Women's Store,Dive Bar,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
9,Scarborough,1,College Stadium,Skating Rink,General Entertainment,Café,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Farmers Market,Dive Bar
10,Scarborough,1,Indian Restaurant,Gaming Cafe,Pet Store,Vietnamese Restaurant,Chinese Restaurant,Latin American Restaurant,Drugstore,Dive Bar,Dog Run,Doner Restaurant


Cluster 3

In [36]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,North York,2,Baseball Field,Home Service,Food Truck,Women's Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
97,North York,2,Baseball Field,Empanada Restaurant,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Women's Store,Dive Bar


Cluster 4

In [37]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Scarborough,3,Park,Playground,Eastern European Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
23,North York,3,Park,Convenience Store,Bank,Women's Store,Electronics Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
40,East York,3,Park,Convenience Store,Metro Station,Women's Store,Electronics Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
50,Downtown Toronto,3,Park,Trail,Playground,Women's Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
98,York,3,Convenience Store,Park,Women's Store,Empanada Restaurant,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store


Cluster 5

In [38]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Central Toronto,4,Garden,Eastern European Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Women's Store,Discount Store


Cluster 6

In [39]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,Central Toronto,5,Bus Line,Swim School,Park,Women's Store,Eastern European Restaurant,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store


Cluster 7

In [40]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 6, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,North York,6,Piano Bar,Women's Store,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant
